In [1]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

**Homework**

- Create a chart to show how the relative share of various playlist_genres has changed over time
- Create a multiple line chart to show the evolution of the average song properties (only those normalized in the range 0-1 ) over time
- Create a small multiple chart with histograms of track popularity for the different genres. Choose an optimal binning
- Compare the distributions of the average song duration for the different genres using a visualization idiom for statistical analysis
- Create a chart showing the how much the attributes (popularity, danceability, energy, loudness, acousticness, instrumentalness, liveness, valence, tempo) are correlated. Use a proper colormap.
- Visualize a scatter plot with songs released in 2010, representing loudness and danceability on the x-y coordinates, dots colored by genres and using a different mark (pre-attentive) to make a specific artist of your interest evident.

In [2]:
songs = pd.read_csv("spotify_songs.csv")